In [ ]:
#|default_exp build_catalogs

# Build Catalogs

> create monthly and catalog tables

In [ ]:
#| eval: false
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import requests
import json
from fastcore.all import L
import pandas as pd
from typing import Any
from urllib.parse import urlparse
from pathlib import Path
import numpy as np
import re
from shapely.geometry import Polygon, box
import geopandas as gpd
import sqlite3 as sql3
from pandas.io import sql 
import sqlalchemy as sqalc

In [ ]:
#| export
from ntlights_damage_assessment.catalogs import *
from ntlights_damage_assessment.items import *

In [ ]:
#| hide
import matplotlib.pyplot as plt

In [ ]:
#| hide
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',120)

In [ ]:
#| export
DATA_DB_DIR = '../data/sqlite'
DB = f'{DATA_DB_DIR}/db.db'

In [ ]:
#| hide
#| eval: false
!mkdir -p ../data/sqlite

In [ ]:
# %%time
# #| eval: false
# catalogs = get_item_catalogs()
# conn = sql3.connect(DB)
# conn.executescript('drop table if exists catalog_items;');
# i = 0
# monthly = catalogs.iloc[i]
# href, folder, baseurl = monthly.href, monthly.folder, monthly.baseurl
# %%time
# items = get_monthly_items(href, folder, baseurl)
# %%time
# count = items.to_sql('catalog_items', conn, if_exists='append', index=False, method='multi')
# print(f'rows added: {count}')
         

In [ ]:
#| export
def build_item_catalogs(catalogs, conn, show=False):
    for i in range(len(catalogs)):
        monthly = catalogs.iloc[i]
        href, folder, baseurl = monthly.href, monthly.folder, monthly.baseurl
        items = get_monthly_items(href, folder, baseurl)
        count = items.to_sql('catalog_items', conn, index=False, if_exists='append', method='multi')
        if show: print(f'{i} rows added: {count}')
         

In [ ]:
create_catalog_items_sql = '''
CREATE TABLE IF NOT EXISTS "catalog_items" (                                                                                
  "href" TEXT,                                                                                                                 
  "base_url" TEXT,
  "folder" TEXT,
  "product_id" TEXT,
  "start_date" TEXT,
  "first_scantime" TEXT,
  "end_scantime" TEXT,
  "orbital_nbr" TEXT,
  "create_datetime" TEXT,
  "data_origin" TEXT,
  "data_domain" TEXT,
  "vflag_file" TEXT,
  "vflag_href" TEXT
);
'''

In [ ]:
#| export
index_on_stem = '''
  create index stem_index on catalog_items(stem);
'''

In [ ]:
#| export
def refresh_item_catalogs(limit=None, verbose=False):
    catalogs = get_item_catalogs()
    conn = sql3.connect(DB)
    conn.executescript('drop table if exists catalog_items;')
    if limit is not None:
        catalogs = catalogs.iloc[:limit]
    build_item_catalogs(catalogs,conn,show=verbose)
    cursor = conn.cursor()
    cursor.execute(index_on_stem)
    
    conn.close()

In [ ]:
%%time
#| eval: false
#| hide
refresh_item_catalogs(verbose=True)

0 rows added: 3645
1 rows added: 3557
2 rows added: 3432
3 rows added: 3620
4 rows added: 3861
5 rows added: 3867
6 rows added: 4017
7 rows added: 3712
8 rows added: 3714
9 rows added: 3786
10 rows added: 3574
11 rows added: 4029
12 rows added: 3757
13 rows added: 3671
14 rows added: 3465
15 rows added: 3621
16 rows added: 3798
17 rows added: 3886
18 rows added: 4023
19 rows added: 3744
20 rows added: 3751
21 rows added: 3788
22 rows added: 3534
23 rows added: 3997
24 rows added: 3764
25 rows added: 3992
26 rows added: 3458
27 rows added: 3620
28 rows added: 3775
29 rows added: 3884
30 rows added: 3938
31 rows added: 3905
32 rows added: 3745
33 rows added: 4033
34 rows added: 3641
35 rows added: 4035
36 rows added: 3902
37 rows added: 4033
38 rows added: 3490
39 rows added: 3625
40 rows added: 4033
41 rows added: 3881
42 rows added: 4026
43 rows added: 3754
44 rows added: 3721
45 rows added: 3780
46 rows added: 3556
47 rows added: 3988
48 rows added: 3758
49 rows added: 3666
50 rows ad